In [ ]:

from Models.MLP.MLPnetwork import MLPNetwork
from Models.MLP.Value import Value
from Trainer import Trainer
data = [
    ([2.0, 3.0], 0),
    ([1.0, 1.5], 0),
    ([2.5, 2.2], 0),
    ([3.0, 0.5], 1),
    ([4.0, 1.0], 1),
    ([3.5, 0.2], 1),
]

# Binary Cross-Entropy loss for sigmoid outputs
def bce_loss(prob, y_true):
    """
    Binary Cross-Entropy Loss for sigmoid outputs.
    
    prob: Value    # model output after sigmoid (a probability)
    y_true: int    # 0 or 1
    """

    # y * log(p)
    term1 = prob.log() if y_true == 1 else Value(0.0)

    # (1 - y) * log(1 - p)
    term2 = (1 - prob).log() if y_true == 0 else Value(0.0)

    # BCE = - (term1 + term2)
    return -(term1 + term2)
    


# creating the model with 1 hidden layer and 1 output
model = MLPNetwork(
    input_dim=2,
    n_neurons=[4, 1],          # [hidden_size, output_size]
    label="toy",
    activation_type="tanh",
    classification="sigmoid"
)

# Training loop (simple SGD)
learning_rate = 0.1
epochs = 200

for epoch in range(epochs):
    total_loss = Value(0.0)
    correct = 0

    for (x_raw, y_true) in data:
        # Forward pass
        probs = model.forward(x_raw)
        loss = bce_loss(probs[0], y_true)

        # Accumulate loss for reporting
        total_loss = total_loss + loss

        # Accuracy
        pred = 1 if probs[0].data > 0.5 else 0
        if pred == y_true:
            correct += 1

        # Backward + update (SGD step)
        model.zero_grad()     # reset grads BEFORE backward
        loss.backward()
        for p in model.parameters():
            p.data -= learning_rate * p.grad

    # Report average loss + accuracy
    if epoch % 20 == 0:
        avg_loss = total_loss.data / len(data)
        acc = correct / len(data)
        print(f"epoch {epoch:3d} | loss={avg_loss:.4f} | acc={acc:.2f}")

# test
print("\nFinal predictions:")
for (x_raw, y_true) in data:
    probs = model.forward(x_raw)
    pred = 1 if probs[0].data > 0.5 else 0
    print(f"x={x_raw}, y_true={y_true}, prob={probs[0].data:.3f}, pred={pred}")

epoch   0 | loss=1.2816 | acc=0.50
epoch  20 | loss=0.0855 | acc=1.00
epoch  40 | loss=0.0409 | acc=1.00
epoch  60 | loss=0.0243 | acc=1.00
epoch  80 | loss=0.0166 | acc=1.00
epoch 100 | loss=0.0124 | acc=1.00
epoch 120 | loss=0.0099 | acc=1.00
epoch 140 | loss=0.0082 | acc=1.00
epoch 160 | loss=0.0069 | acc=1.00
epoch 180 | loss=0.0060 | acc=1.00

Final predictions:
x=[2.0, 3.0], y_true=0, prob=0.004, pred=0
x=[1.0, 1.5], y_true=0, prob=0.005, pred=0
x=[2.5, 2.2], y_true=0, prob=0.007, pred=0
x=[3.0, 0.5], y_true=1, prob=0.993, pred=1
x=[4.0, 1.0], y_true=1, prob=0.995, pred=1
x=[3.5, 0.2], y_true=1, prob=0.995, pred=1


In [1]:
from Models.MLP import MLPNetwork
from Trainer.trainer import Trainer
data = [
    ([2.0, 3.0], [0,1]),
    ([1.0, 1.5], [0,1]),
    ([2.5, 2.2], [0,1]),
    ([3.0, 0.5], [0,1]),
    ([4.0, 1.0], [0,1]),
    ([3.5, 0.2], [0,1]),
]
predictors = []
labels = []

for x,y in data:
    predictors.append(x)
    labels.append(y)

model = MLPNetwork(input_dim=2, n_neurons=[4, 2], label="toy", activation_type="tanh", classification="none")
trainer = Trainer(model=model, epochs=100)
trainer.fit(predictors, labels)
trainer.test(predictors, labels)

Trainer initialized for regression task using LinearLoss loss.
Epoch 1/100, Loss: 3.1873
Epoch 20/100, Loss: 0.0686
Epoch 40/100, Loss: 0.0203
Epoch 60/100, Loss: 0.0099
Epoch 80/100, Loss: 0.0063
Epoch 100/100, Loss: 0.0046
Test Loss: 0.0044


0.004350684529580903

In [1]:
import random

def make_toy_data(n_per_class=100):
    X_real = []
    y_labels = []

    for _ in range(n_per_class):
        # class 0: around (-1, -1)
        x1 = -1 + random.gauss(0, 0.2)
        x2 = -1 + random.gauss(0, 0.2)
        X_real.append([x1, x2])
        y_labels.append(0)

    for _ in range(n_per_class):
        # class 1: around (1, 1)
        x1 = 1 + random.gauss(0, 0.2)
        x2 = 1 + random.gauss(0, 0.2)
        X_real.append([x1, x2])
        y_labels.append(1)

    return X_real, y_labels

In [2]:
from Models.MLPGenerator import MLPGenerator
from Trainer.trainer import Trainer
import random

def make_toy_data(n_per_class=100):
    X_real = []
    y_labels = []

    for _ in range(n_per_class):
        # class 0: around (-1, -1)
        x1 = -1 + random.gauss(0, 0.2)
        x2 = -1 + random.gauss(0, 0.2)
        X_real.append([x1, x2])
        y_labels.append(0)

    for _ in range(n_per_class):
        # class 1: around (1, 1)
        x1 = 1 + random.gauss(0, 0.2)
        x2 = 1 + random.gauss(0, 0.2)
        X_real.append([x1, x2])
        y_labels.append(1)

    return X_real, y_labels

def to_one_hot(label, num_classes):
    return [1 if i == label else 0 for i in range(num_classes)]

def build_generator_training_data(X_real, y_labels, latent_dim, num_classes):
    X_train = []   # inputs to generator
    y_train = []   # targets (real x)

    for x, y in zip(X_real, y_labels):
        # sample random noise z
        z = [random.uniform(-1, 1) for _ in range(latent_dim)]
        # one-hot encode label
        y_vec = to_one_hot(y, num_classes)
        # concat [z, y_onehot] as input
        inp = z + y_vec

        X_train.append(inp)
        y_train.append(x)

    return X_train, y_train
# hyperparameters
latent_dim  = 4          # size of noise vector z
num_classes = 2          # y in {0, 1}
cond_dim    = num_classes
output_dim  = 2          # x is 2D
hidden_sizes = [16, 16]  # small MLP

# create generator model
gen_model = MLPGenerator(
    latent_dim=latent_dim,
    cond_dim=cond_dim,
    output_dim=output_dim,
    hidden_sizes=hidden_sizes,
    activation_type="tanh"
)

# create trainer (regression, so classification="none" in model)
trainer = Trainer(
    model=gen_model,
    optimizer=None,      # uses default SGD in your __init__
    loss_fn=None,        # uses default LinearLoss in your __init__
    epochs=50
)

Trainer initialized for regression task using LinearLoss loss.


In [3]:
# 1. real data
X_real, y_labels = make_toy_data(n_per_class=200)

# 2. build (input, target) pairs for generator
X_train, y_train = build_generator_training_data(
    X_real, y_labels,
    latent_dim=latent_dim,
    num_classes=num_classes
)

# 3. train
trainer.fit(X_train, y_train)

Epoch 1/50, Loss: 0.2782
Epoch 20/50, Loss: 0.0442
Epoch 40/50, Loss: 0.0426


In [5]:
def sample_z(latent_dim):
    return [random.uniform(-1, 1) for _ in range(latent_dim)]

# generate 5 new samples for class 0
print("Synthetic samples for class 0:")
for _ in range(5):
    z = sample_z(latent_dim)
    y_vec = to_one_hot(0, num_classes)      # condition on class 0
    x_hat = gen_model.generate(z, y_vec)
    print(x_hat)

# generate 5 new samples for class 1
print("\nSynthetic samples for class 1:")
for _ in range(5):
    z = sample_z(latent_dim)
    y_vec = to_one_hot(1, num_classes)      # condition on class 1
    x_hat = gen_model.generate(z, y_vec)
    print(x_hat)

Synthetic samples for class 0:
[Value: -1.0559758216501918, Grad: 0, Value: -0.9542930327942515, Grad: 0]
[Value: -0.9809952181716437, Grad: 0, Value: -1.0358120493219025, Grad: 0]
[Value: -1.0725440286621213, Grad: 0, Value: -0.9512467152491785, Grad: 0]
[Value: -1.0064335722720055, Grad: 0, Value: -1.0117034090288402, Grad: 0]
[Value: -0.9000556539326645, Grad: 0, Value: -1.049317561478556, Grad: 0]

Synthetic samples for class 1:
[Value: 0.9608756327356689, Grad: 0, Value: 0.9623279563241465, Grad: 0]
[Value: 1.0562431388054598, Grad: 0, Value: 0.9640717531464561, Grad: 0]
[Value: 1.014131203869769, Grad: 0, Value: 0.9649806802962615, Grad: 0]
[Value: 1.1632670713306237, Grad: 0, Value: 0.8835455789731771, Grad: 0]
[Value: 1.0035622775327868, Grad: 0, Value: 0.9617033974953391, Grad: 0]
